# Семантическая Сегментация. Часть 2.

ASPP - ATROUS SPATIAL PYRAMID POOLING

Основная идея в том, что кроме частей энкодер(уменьшение пространственного измерения) и декодер(увеличение) в середине был блок ASPP, суть которого в применении несколько параллельных делатеционных сверток(увеличение рецептивного поля)

In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [ ]:
import tensorflow as tf

## Модель ASPP

In [ ]:
# сам ASPP блок
class ASPPBlock(tf.keras.Model):
  # Свертка
    def __init__(self):
        super().__init__()
        # Свертка 1 - поканальная смесь всех наших каналов
        self.conv1 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')
        # Группа делатеционных сверток, в котором dilation_rate=6 - шаг разброса между пикселями(условно)
        self.conv2 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=6, padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=12, padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=18, padding='same', activation='relu')
        # С помощью свертки 1х1 меняем кол-во каналов(поканально перешивая карты признаков) и получаем на выходе 256 карт признаков
        self.conv5 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')
  # Инференс
    def call(self, inp, is_training=False):
        out1 = self.conv1(inp)
        out2 = self.conv2(inp)
        out3 = self.conv3(inp)
        out4 = self.conv4(inp)
        # Конкатенируем результаты по канальному измерению
        out = tf.concat([out1, out2, out3, out4], axis=3)
        out = self.conv5(out)
        return out
    
class ASPPNet(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # Encoder
        self.conv1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv5 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv6 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv7 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv8 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv9 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv10 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        # Decoder
        self.conv11 = tf.keras.layers.Conv2D(48, (1, 1), padding='same', activation='relu')
        self.conv12 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv13 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        # Свертка для выходной карты признаков
        self.conv14 = tf.keras.layers.Conv2D(1, (1, 1), padding='same', activation=None)

        self.maxpool = tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same')
        # Вызываем наш ASPP слой, чтобы инициализировать веса
        self.aspp = ASPPBlock()

    def call(self, x):

        out = self.conv1(x)
        out = self.conv2(out)
        out = self.maxpool(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.maxpool(out)
        out = self.conv5(out)
        out = self.conv6(out)
        # Сохраняем промежуточный тензор, чтобы потом через Skip Connection подать его в конец
        out_enc_mid = out
        out = self.maxpool(out)
        out = self.conv7(out)
        out = self.conv8(out)
        out = self.maxpool(out)
        out = self.conv9(out)
        out = self.conv10(out)
        # Дошли до самого глубокого состояния энкодера и применяем ASPP
        out = self.aspp(out)
        # Очень просто повышаем разрешение картинок(на деле так нельзя) до размеров промежуточного тензора, где [1:3] означает 2 пространсвенных измерения(1 - ширина, 2 - высота)
        out = tf.image.resize(out, tf.shape(out_enc_mid)[1:3], tf.image.ResizeMethod.BILINEAR)
        # Дополнительная свертка 1х1, задача которой привести тензор к фиксированному кол-ву каналов(в данном с случае до 48)
        out_enc_mid = self.conv11(out_enc_mid)
        # Пробрасываем Skip Connection
        out = tf.concat([out, out_enc_mid], axis=3)

        out = self.conv12(out)
        out = self.conv13(out)
        out = self.conv14(out)
        # Ресайз до размеров Х - входной картинки
        out = tf.image.resize(out, tf.shape(x)[1:3], tf.image.ResizeMethod.BILINEAR)
        # Сигмоидная функция для вероятности принадлежности пикселей классам
        out = tf.nn.sigmoid(out)
        return out
    
model = ASPPNet()